In [42]:
import requests
import re
import pandas as pd
import folium
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import json

In [43]:
#zipcodes in LA
f = open('inputZips.txt', 'r')
allZips = [z.replace('\n','') for z in f.readlines()]

In [44]:
def processResponse(r):
    #parse out each store's info
    stores = re.findall(r'"storeNumber":.*?"slug"', r)
    storeInfo = []
    for store in stores:
        #parse out info about each store
        storeInfo.append(list(re.findall(r'"storeNumber":"(.*?)".*?"name":"(.*?)".*?"latitude":(.*?),.*?"longitude":(.*?)}.*?"city":"(.*?)".*?"countrySubdivisionCode":"(.*?)".*?"postalCode":"(.*?)"', store)[0]))
    return storeInfo

In [45]:
allStores = []
for idx,nonSplitLine in enumerate(allZips):
    line = nonSplitLine.split(",")
    if idx%10 == 0:
        print(idx,'/',len(allZips))
    #search for 100 stores centered in given zip
    # r = requests.get('https://www.starbucks.com/store-locator?map=34.216464,-118.656046,11z&place='+z)
    # 1923,Danvers,Massachusetts,MA,Essex,42.5694,-70.9425
    post = line[0]
    lat = line[1]
    lon = line[2]
    urlll = 'https://www.starbucks.com/store-locator?map='+lat+','+lon+'z&place='+post
    r = requests.get(urlll)
    if r.status_code != 200:
        raise SystemExit
    storeInfoList = processResponse(r.text)
    for storeInfo in storeInfoList:
        storeInfo[6] = storeInfo[6][:5]
    allStores += storeInfoList

['501', '40.8154', '-73.0451', 'Holtsville', 'New York', 'NY', 'Suffolk']
0 / 5
https://www.starbucks.com/store-locator?map=40.8154,-73.0451z&place=501
['544', '40.8154', '-73.0451', 'Holtsville', 'New York', 'NY', 'Suffolk']
https://www.starbucks.com/store-locator?map=40.8154,-73.0451z&place=544
['1001', '42.0702', '-72.6227', 'Agawam', 'Massachusetts', 'MA', 'Hampden']
https://www.starbucks.com/store-locator?map=42.0702,-72.6227z&place=1001
['1002', '42.3671', '-72.4646', 'Amherst', 'Massachusetts', 'MA', 'Hampshire']
https://www.starbucks.com/store-locator?map=42.3671,-72.4646z&place=1002
['1003', '42.3919', '-72.5248', 'Amherst', 'Massachusetts', 'MA', 'Hampshire']
https://www.starbucks.com/store-locator?map=42.3919,-72.5248z&place=1003


In [46]:
#construct non-duplicated list of stores
seenStoreIds = []
laStores = []
for store in allStores:
    if store[0] in seenStoreIds:
        continue
    else:
        laStores.append(store)
        seenStoreIds.append(store[0])

In [47]:
#define columns of csv
dfSbux = pd.DataFrame(columns=['id', 'strLocation', 'latitude', 'longitude', 'city', 'state', 'zip'])

#store data
for i,col in enumerate(dfSbux.columns):
    dfSbux[col] = [item[i] for item in laStores]

#cast the latitude and longitude as floats
dfSbux.latitude = dfSbux.latitude.apply(lambda x: float(x))
dfSbux.longitude = dfSbux.longitude.apply(lambda x: float(x))

#save to csv
dfSbux.to_csv('starbucksInUS.csv', index=False)

In [48]:
# dfSbux = pd.DataFrame(columns=['id', 'strLocation', 'latitude', 'longitude', 'city', 'state', 'zip'])
# for i,col in enumerate(dfSbux.columns):
#     dfSbux[col] = [item[i] for item in keepLAStores]

In [49]:
# dfSbux.latitude = dfSbux.latitude.apply(lambda x: float(x))
# dfSbux.longitude = dfSbux.longitude.apply(lambda x: float(x))

In [50]:
# laMap = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=10)
# for i,row in dfSbux.iterrows():
#     folium.CircleMarker((row.latitude,row.longitude), radius=3, weight=2, fill_color='red', fill_opacity=.9).add_to(laMap)

In [51]:
# folium.GeoJson(laArea).add_to(laMap)